In [2]:
!git config global user.name "maryamSayagh"
!git config global user.email "maryamsayagho@gmail.com"
!git clone "https://github.com/maryamSayagh/HRreview.git"

fatal: not in a git directory
fatal: not in a git directory
Cloning into 'HRreview'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 136 (delta 48), reused 102 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (136/136), 5.80 MiB | 11.14 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [3]:
!pip install dotenv

In [4]:
import re
import os
import pandas as pd
import numpy as np
import sys
from dotenv import load_dotenv

#from papers_table import column_name


In [5]:
load_dotenv()

!cd /content/HRreview/review/data/Journals/Scopus


In [6]:
#os.chdir("..")  # Moves up one folder
print("Current working directory:", os.getcwd())

Current working directory: /content


In [7]:
csv_path = "../review/data/Journals/Scopus/ext_ist_Jul_2025_Scopus_Sources_Jul.csv"



In [8]:
#os.chdir("C:/Users/Lenovo/PycharmProjects/HRreview/review/data/Journals/Scopus/ext_list_Jul_2025_Scopus_Sources_Jul.csv")

In [9]:
os.getcwd()

'/content'

In [10]:




scopus = pd.read_csv("/content/HRreview/review/data/Journals/Scopus/ext_list_Jul_2025_Scopus_Sources_Jul.csv",sep=";",encoding='latin1')


In [11]:
scopus.head(5)


,Sourcerecord ID,Source Title,ISSN,EISSN,Active or Inactive,Coverage,Titles Discontinued by Scopus Due to Quality Issues,Article Language in Source (Three-Letter ISO Language Codes),Medline-sourced Title? (See additional details under separate tab.),Open Access Status,...,2700\nMedicine,2800\nNeuroscience,2900\nNursing,"3000\nPharmacology, Toxicology and Pharmaceutics",3100\nPhysics and Astronomy,3200\nPsychology,3300\nSocial Sciences,3400\nVeterinary,3500\nDentistry,3600\nHealth Professions
0,21101170611,@GRH,20349130,22959149,Active,2023-2024,NaN,ENG; FRE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21100902951,[Hifu] Skin research,00181390,NaN,Inactive,1959-2001,NaN,JPN,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22012,[Nippon koshu eisei zasshi] Japanese journal o...,05461766,NaN,Inactive,1990-2016,NaN,NaN,Medline-unique,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26063,[Rinsho ketsueki] The Japanese journal of clin...,04851439,18820824,Inactive,1972-2016,NaN,NaN,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40160,[Zasshi] [Journal]. Nihon Kyobu Geka Gakkai,03694739,NaN,Inactive,1972-1997,NaN,JPN,NaN,NaN,...,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
scopus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47758 entries, 0 to 47757
Data columns (total 52 columns):
 #   Column                                                               Non-Null Count  Dtype 
---  ------                                                               --------------  ----- 
 0   Sourcerecord ID                                                      47758 non-null  int64 
 1   Source Title                                                         47758 non-null  object
 2   ISSN                                                                 42071 non-null  object
 3   EISSN                                                                28372 non-null  object
 4   Active or Inactive                                                   47758 non-null  object
 5   Coverage                                                             47757 non-null  object
 6   Titles Discontinued by Scopus Due to Quality Issues                  862 non-null    object
 7   Article Langu

In [13]:
#this one clean the parsing of the date, ask gpt not me
#we are only selected journals that are active after 2010, because that when AI-Human interaction rised
def keep_journals_active_after(df, year_col='Coverage', min_year=2010):
    def extract_years(text):
        if not isinstance(text, str):
            return []
        years = set()
        # Normalize en dash to hyphen
        text = text.replace('–', '-')
        parts = [p.strip() for p in text.split(';')]
        for p in parts:
            if '-' in p:
                match = re.findall(r'(\d{4})-(\d{4})', p)
                if match:
                    start, end = map(int, match[0])
                    years.update(range(start, end + 1))
            else:
                match = re.findall(r'\d{4}', p)
                for y in match:
                    years.add(int(y))
        return years

    def is_recent(year_set):
        return any(y >= min_year for y in year_set)

    df = df.copy()
    df['__years'] = df[year_col].apply(extract_years)
    df = df[df['__years'].apply(is_recent)].drop(columns='__years')
    return df

scopus=keep_journals_active_after(scopus, year_col='Coverage', min_year=2010)

In [14]:

# Use this if you are only interested in journals that publish articles in English
scopus = scopus[scopus['Article Language in Source (Three-Letter ISO Language Codes)'].str.contains("ENG", na=False)]


In [15]:

# Your utility to read context
def read_txt(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

# Build context exactly like your snippet
context = read_txt("/content/HRreview/review/data/about.txt")
keywords = [
    "Human in the loop",
    "AI-human interaction",
    "Decision Making",
    "AI recommendation",
    "Trust in recommendation",
    "Explanable Artificial inteligence",
    "Artificial intelligence Bias",
    "Operational Analytics Decision",
    "AI assisted decision making",
    "Trust in automation",
    "Automation bias",
    "Human-AI Collaboration",
    "HAIC Evaluation"
]
context += "\nThese are keywords referring to the subject we are studying: [" + ", ".join(keywords) + "]"


In [16]:
scopus

,Sourcerecord ID,Source Title,ISSN,EISSN,Active or Inactive,Coverage,Titles Discontinued by Scopus Due to Quality Issues,Article Language in Source (Three-Letter ISO Language Codes),Medline-sourced Title? (See additional details under separate tab.),Open Access Status,...,2700\nMedicine,2800\nNeuroscience,2900\nNursing,"3000\nPharmacology, Toxicology and Pharmaceutics",3100\nPhysics and Astronomy,3200\nPsychology,3300\nSocial Sciences,3400\nVeterinary,3500\nDentistry,3600\nHealth Professions
0,21101170611,@GRH,20349130,22959149,Active,2023-2024,NaN,ENG; FRE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,21101194408,20 & 21: Revue d'Histoire,2649664X,26496100,Active,2022-2024,NaN,ENG; FRE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,18500162600,21st Century Music,15343219,NaN,Inactive,2002-2011,NaN,ENG,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,21100404576,2D Materials,NaN,20531583,Active,2014-2025,NaN,ENG,NaN,NaN,...,NaN,NaN,NaN,NaN,Physics and Astronomy,NaN,NaN,NaN,NaN,NaN
9,21100447128,3 Biotech,2190572X,21905738,Active,2011-2025,NaN,ENG,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47747,23139,Zuckerindustrie,03448657,NaN,Inactive,2017-2025; 1996-2013; 1987-1989; 1977-1985,NaN,ENG; GER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47749,21100403905,Zutot,15717283,18750214,Active,2008-2025; 2001-2004,NaN,ENG,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47751,21100317006,Zwingliana,02544407,NaN,Inactive,2013-2019; 2002; 1981,NaN,ENG; FRE; GER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47753,28883,Zygon,05912385,14679744,Active,1966-2024,NaN,ENG,NaN,Unpaywall Open Access,...,NaN,NaN,NaN,NaN,NaN,NaN,Social Sciences,NaN,NaN,NaN



 removing irrelevant fields/columns


In [17]:
#getting rid of columns that are not relevant
non_category_cols = [
    '1000 \nGeneral', '1100\nAgricultural and Biological Sciences', '1200\nArts and Humanities',
    '1300\nBiochemistry, Genetics and Molecular Biology',
    '1500\nChemical Engineering', '1600\nChemistry',
    '1900\nEarth and Planetary Sciences',  '2100\nEnergy',
     '2300\nEnvironmental Science', '2400\nImmunology and Microbiology',
    '2500\nMaterials Science',  '2700\nMedicine',  '2900\nNursing',
    '3000\nPharmacology, Toxicology and Pharmaceutics', '3100\nPhysics and Astronomy', '3200\nPsychology',
    '3400\nVeterinary', '3500\nDentistry', '3600\nHealth Professions',
     'Top level:\n\nHealth Sciences', 'Top level:\n\nPhysical Sciences', 'Top level:\n\nLife Sciences',
]
Complete=['Top level:\n\nLife Sciences', 'Top level:\n\nSocial Sciences',
        'Top level:\n\nHealth Sciences',
       '1000 \nGeneral', '1100\nAgricultural and Biological Sciences',
       '1200\nArts and Humanities',
       '1300\nBiochemistry, Genetics and Molecular Biology',
       '1400\nBusiness, Management and Accounting',
       '1500\nChemical Engineering', '1600\nChemistry',
       '1700\nComputer Science', '1800\nDecision Sciences',
       '1900\nEarth and Planetary Sciences',
       '2000\nEconomics, Econometrics and Finance', '2100\nEnergy',
       '2200\nEngineering', '2300\nEnvironmental Science',
       '2400\nImmunology and Microbiology', '2500\nMaterials Science',
       '2600\nMathematics', '2700\nMedicine', '2800\nNeuroscience',
       '2900\nNursing', '3000\nPharmacology, Toxicology and Pharmaceutics',
       '3100\nPhysics and Astronomy', '3200\nPsychology',
       '3300\nSocial Sciences', '3400\nVeterinary', '3500\nDentistry',
       '3600\nHealth Professions']


category_cols = [col for col in Complete if col not in non_category_cols]
# I kept most of the top level columns because those are container categories they may hold a lot within

In [18]:
mask_relevant = scopus[category_cols].notna().any(axis=1)
mask_unclassified = scopus[Complete].isna().all(axis=1)

#Keep rows where the journal is either in a relevant category OR not classified at all:
scopus_clean = scopus[mask_relevant | mask_unclassified]

In [19]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 kB 9.3 MB/s eta 0:00:00


In [20]:
from google.colab import userdata
key=userdata.get('mistral')

In [21]:

import os


scripts_path = "/content/HRreview/review/Scripts"
os.chdir(scripts_path)
import journal_filter_gpt
load_dotenv()




from mistralai import Mistral
#key= os.environ.get('mistral')
client = Mistral(api_key=key)


In [ ]:


scopus_clean_subs = [d.reset_index(drop=True) for d in np.array_split(scopus_clean, 30)]
print(f"Created {len(scopus_clean_subs)} sub-datasets with sizes {[len(s) for s in scopus_clean_subs]}")

li = []
for set in scopus_clean_subs:
    f = journal_filter_gpt.filter_journal_dataframe(
        df=set,
        client=client,
        column_names=["Source Title"],
        context_text=context,
        strategy="dict",
        use_chunks=False,
        chunk_size=1500,
        model="mistral-small",
        use_tqdm=True
    )
    li.append(f)

print(li)


Created 30 sub-datasets with sizes [638, 638, 638, 638, 638, 638, 638, 638, 638, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637]
[GPT Filter] Starting filtering for column 'Source Title' as a single chunk...
Chunk size: 638
[Model Filter] Exception occurred: unknown extension ?\: at position 4. 
[Model Filter] Returning partially filtered DataFrame.
[GPT Filter] Starting filtering for column 'Source Title' as a single chunk...
Chunk size: 638


In [ ]:
li[9].head()

In [ ]:

# Normalize possible returns (list/dict/DataFrame) but keep logic minimal
if isinstance(f, list):
    accepted_titles = set(map(str, f))
elif isinstance(f, dict):
    # Try common keys; if your function returns dict, adjust here if needed
    for k in ("keep", "accepted", "titles", "result"):
        if k in f and isinstance(f[k], (list, set, tuple)):
            accepted_titles = set(map(str, f[k]))
            break
    else:
        raise ValueError("Unexpected dict structure from LLM filter. Please return a list of titles.")
elif isinstance(f, pd.DataFrame) and 'Source Title' in f.columns:
    accepted_titles = set(map(str, f['Source Title'].dropna().unique().tolist()))
else:
    raise TypeError("Unexpected return type from LLM filter. Expected list/dict/DataFrame with 'Source Title'.")

print(f"Accepted titles: {len(accepted_titles)}")
list(sorted(list(accepted_titles))[:10])


In [ ]:

# Filter the original scopus DataFrame by accepted titles and save
scopus_filtered = scopus[scopus['Source Title'].astype(str).isin(accepted_titles)].copy()
scopus_filtered.reset_index(drop=True, inplace=True)

out_path = "data/Journals/Scopus/Scopus_list_sources_llm_filtered.csv"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
scopus_filtered.to_csv(out_path, index=False)
print(f"Saved filtered journals to: {out_path} (rows: {len(scopus_filtered)})")
scopus_filtered.head(5)